In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.context import SQLContext 
import matplotlib.pyplot as plt

# Creating a Spark Session

In [ ]:
sc=SparkContext()
sqlContext=SQLContext(sc)
spark=SparkSession(sc)

In [5]:
import re
import pandas as pd
import glob

# Inserting every log file having a similar pattern with extension .gz

In [6]:
raw_files=glob.glob('/home/mav_27/Project/Data/nginx-feb2023/nginx/*.gz')


In [7]:
raw_files

['/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230217.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/error.log-20230220.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230214.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230215.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230222.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/error.log-20230217.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230213.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/access.log-20230217.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230219.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230218.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/access.log-20230213.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/access.log-20230216.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/proxy.log-20230220.gz',
 '/home/mav_27/Project/Data/nginx-feb2023/nginx/access.log-20

# Creating a Base Dataframe which contains all raw files

In [9]:
base_df=spark.read.text(raw_files)

In [10]:
base_df.printSchema()

root
 |-- value: string (nullable = true)



In [11]:
base_df.count()

6218

In [12]:
len(base_df.columns)

1

In [201]:
base_df.show(10,truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                        |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|143.244.50.172 - - [16/Feb/2023:03:28:4

# Taking a Sample data from Base Dataframe for data parsing

In [202]:
sample_logs=[items['value'] for items in base_df.take(15)]
sample_logs

['143.244.50.172 - - [16/Feb/2023:03:28:45 +0530] "GET /config/getuser?index=0 HTTP/1.1" 400 248 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0"',
 '164.90.235.116 - - [16/Feb/2023:04:11:34 +0530] "GET / HTTP/1.1" 200 5952 "http://14.139.152.12/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"',
 '66.249.69.126 - - [16/Feb/2023:04:39:52 +0530] "GET /robots.txt HTTP/1.1" 404 146 "-" "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"',
 '66.249.69.126 - - [16/Feb/2023:04:39:52 +0530] "GET /assets/img/favicon.png HTTP/1.1" 200 491 "-" "Googlebot-Image/1.0"',
 '185.221.219.172 - - [16/Feb/2023:05:06:47 +0530] "GET /.git/config HTTP/1.1" 404 548 "-" "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.2; Trident/5.0)"',
 '66.249.69.101 - - [16/Feb/2023:05:24:52 +0530] "GET /apim/devportal/site/public/images/logo.svg HTTP/1.1" 304 0 "-" "Googlebot-Image/1.0"',
 '52.167.144

In [203]:
host_pattern=r'(^\S+\.[\S+\.]+\S+)\s'
hosts=[re.search(host_pattern,items).group(1) 
       if re.search(host_pattern,items) 
       else 'no match' 
       for items in sample_logs]

In [204]:
hosts

['143.244.50.172',
 '164.90.235.116',
 '66.249.69.126',
 '66.249.69.126',
 '185.221.219.172',
 '66.249.69.101',
 '52.167.144.90',
 '152.89.196.211',
 '114.119.133.53',
 '198.20.69.98',
 '198.20.69.98',
 '198.20.69.98',
 '198.20.69.98',
 '198.20.69.98',
 '198.20.69.98']

In [205]:
time_stamp_pattern=r'\[(\d{2}/\w{3}/\d{4})'
date=[re.search(time_stamp_pattern,items).group(1)
            for items in sample_logs]
date

['16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023',
 '16/Feb/2023']

In [206]:
time_pattern=r':\d{2}:\d{2}:\d{2}'
time_raw=[re.search(time_pattern,items.strip(':')).group()
            for items in sample_logs]
time_raw
# time=[]
# for i in time_raw:
#     time.append(i.lstrip(':'))
# time    

[':03:28:45',
 ':04:11:34',
 ':04:39:52',
 ':04:39:52',
 ':05:06:47',
 ':05:24:52',
 ':06:26:16',
 ':07:00:15',
 ':07:01:52',
 ':07:26:00',
 ':07:26:16',
 ':07:26:20',
 ':07:26:21',
 ':07:26:24',
 ':07:26:30']

In [207]:
method_pattern=r'\"(\S+)\s(\S+)\s*(\S*)\"'
method_uri=[re.search(method_pattern,items).group()
           for items in sample_logs]

In [208]:
method_uri

['"GET /config/getuser?index=0 HTTP/1.1"',
 '"GET / HTTP/1.1"',
 '"GET /robots.txt HTTP/1.1"',
 '"GET /assets/img/favicon.png HTTP/1.1"',
 '"GET /.git/config HTTP/1.1"',
 '"GET /apim/devportal/site/public/images/logo.svg HTTP/1.1"',
 '"GET / HTTP/1.1"',
 '"GET /vendor/phpunit/phpunit/src/Util/PHP/eval-stdin.php HTTP/1.1"',
 '"GET /robots.txt HTTP/1.1"',
 '"GET / HTTP/1.1"',
 '"-" "-"',
 '"-" "-"',
 '"-" "-"',
 '"-" "-"',
 '"GET /robots.txt HTTP/1.1"']

In [209]:
status_pattern=r'\s(\d{3})\s'
status_code=[re.search(status_pattern,items).group(1)
            for items in sample_logs]

In [210]:
status_code

['400',
 '200',
 '404',
 '200',
 '404',
 '304',
 '200',
 '404',
 '404',
 '200',
 '400',
 '400',
 '400',
 '400',
 '404']

In [211]:
content_size_pattarn=r'\s(\d+) "'
content_size=[re.search(content_size_pattarn,items).group(1)
              for items in sample_logs]

In [212]:
content_size

['248',
 '5952',
 '146',
 '491',
 '548',
 '0',
 '5952',
 '548',
 '146',
 '5952',
 '0',
 '0',
 '0',
 '0',
 '146']

# Creating a clean dataframe with parsed data

In [213]:
from pyspark.sql.functions import regexp_extract

In [214]:
logs_df=base_df.select(regexp_extract('value',host_pattern,1).alias('Host'),
                       regexp_extract('value',time_stamp_pattern,1).alias('Date'),
                       regexp_extract('value',time_pattern,0).alias('Time'),
                       regexp_extract('value',method_pattern,1).alias('Method'),
                       regexp_extract('value',method_pattern,2).alias('Endpoint'),
                       regexp_extract('value',method_pattern,3).alias('Protocol'),
                       regexp_extract('value',status_pattern,1).alias("Status Code"),
                       regexp_extract('value',content_size_pattarn,1).cast('integer').alias('Content Size'))

In [215]:
logs_df.show(10,truncate=True)

+---------------+-----------+---------+------+--------------------+--------+-----------+------------+
|           Host|       Date|     Time|Method|            Endpoint|Protocol|Status Code|Content Size|
+---------------+-----------+---------+------+--------------------+--------+-----------+------------+
| 143.244.50.172|16/Feb/2023|:03:28:45|   GET|/config/getuser?i...|HTTP/1.1|        400|         248|
| 164.90.235.116|16/Feb/2023|:04:11:34|   GET|                   /|HTTP/1.1|        200|        5952|
|  66.249.69.126|16/Feb/2023|:04:39:52|   GET|         /robots.txt|HTTP/1.1|        404|         146|
|  66.249.69.126|16/Feb/2023|:04:39:52|   GET|/assets/img/favic...|HTTP/1.1|        200|         491|
|185.221.219.172|16/Feb/2023|:05:06:47|   GET|        /.git/config|HTTP/1.1|        404|         548|
|  66.249.69.101|16/Feb/2023|:05:24:52|   GET|/apim/devportal/s...|HTTP/1.1|        304|           0|
|  52.167.144.90|16/Feb/2023|:06:26:16|   GET|                   /|HTTP/1.1|      

In [216]:
logs_df.count()

6218

In [217]:
import ipaddress

In [218]:
(base_df.filter(base_df['value'].isNull()).count())

0

In [219]:
bad_row_df=logs_df.filter(logs_df['Host'].isNull() |
                          logs_df['Date'].isNull() |
                          logs_df['Time'].isNull() |
                          logs_df['Method'].isNull() |
                          logs_df['Endpoint'].isNull() |
                          logs_df['Protocol'].isNull() |
                          logs_df['Status Code'].isNull() |
                          logs_df['Content Size'].isNull())

In [220]:
bad_row_df.count()

482

# Converting the Spark DataFrame to Pandas DataFrame

In [221]:
logs_df=logs_df.toPandas()

In [222]:
logs_df.dtypes

Host             object
Date             object
Time             object
Method           object
Endpoint         object
Protocol         object
Status Code      object
Content Size    float64
dtype: object

In [223]:
logs_df.head()

,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size
0,143.244.50.172,16/Feb/2023,:03:28:45,GET,/config/getuser?index=0,HTTP/1.1,400,248.0
1,164.90.235.116,16/Feb/2023,:04:11:34,GET,/,HTTP/1.1,200,5952.0
2,66.249.69.126,16/Feb/2023,:04:39:52,GET,/robots.txt,HTTP/1.1,404,146.0
3,66.249.69.126,16/Feb/2023,:04:39:52,GET,/assets/img/favicon.png,HTTP/1.1,200,491.0
4,185.221.219.172,16/Feb/2023,:05:06:47,GET,/.git/config,HTTP/1.1,404,548.0


In [224]:
time=[]
for i in logs_df.Time:
       time.append(i.strip(':'))

In [225]:
logs_df['Time']=time

In [226]:
logs_df

,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size
0,143.244.50.172,16/Feb/2023,03:28:45,GET,/config/getuser?index=0,HTTP/1.1,400,248.0
1,164.90.235.116,16/Feb/2023,04:11:34,GET,/,HTTP/1.1,200,5952.0
2,66.249.69.126,16/Feb/2023,04:39:52,GET,/robots.txt,HTTP/1.1,404,146.0
3,66.249.69.126,16/Feb/2023,04:39:52,GET,/assets/img/favicon.png,HTTP/1.1,200,491.0
4,185.221.219.172,16/Feb/2023,05:06:47,GET,/.git/config,HTTP/1.1,404,548.0
...,...,...,...,...,...,...,...,...
6213,,,,GET,/sitemaps.xml,HTTP/1.1,,NaN
6214,,,,GET,/robots.txt,HTTP/1.1,,NaN
6215,,,,GET,/favicon.ico,HTTP/1.1,,NaN
6216,,,,GET,/owa/auth/logon.aspx?url=https%3a%2f%2f1%2fecp%2f,HTTP/1.1,,NaN


In [227]:
logs_df=logs_df.dropna()

In [228]:
logs_df

,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size
0,143.244.50.172,16/Feb/2023,03:28:45,GET,/config/getuser?index=0,HTTP/1.1,400,248.0
1,164.90.235.116,16/Feb/2023,04:11:34,GET,/,HTTP/1.1,200,5952.0
2,66.249.69.126,16/Feb/2023,04:39:52,GET,/robots.txt,HTTP/1.1,404,146.0
3,66.249.69.126,16/Feb/2023,04:39:52,GET,/assets/img/favicon.png,HTTP/1.1,200,491.0
4,185.221.219.172,16/Feb/2023,05:06:47,GET,/.git/config,HTTP/1.1,404,548.0
...,...,...,...,...,...,...,...,...
5731,154.12.227.174,19/Feb/2023,02:48:08,POST,/core/.env,HTTP/1.1,404,146.0
5732,154.12.227.174,19/Feb/2023,02:48:10,GET,/core/.env,HTTP/1.1,404,146.0
5733,154.12.227.174,19/Feb/2023,02:48:12,POST,/,HTTP/1.1,405,150.0
5734,154.12.227.174,19/Feb/2023,02:48:13,POST,/core/.env,HTTP/1.1,404,146.0


In [229]:
ipadd=[]
for i in logs_df.Host:
    ipadd.append(int(ipaddress.IPv4Address(i)))

In [230]:
logs_df.Host=ipadd

/tmp/ipykernel_185/2148316297.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_df.Host=ipadd


In [231]:
logs_df.Method.value_counts()

GET                                                                                                                                      3467
-"                                                                                                                                        789
POST                                                                                                                                      707
CONNECT                                                                                                                                   596
PRI                                                                                                                                        40
                                                                                                                                         ... 
\x16\x03\x01\x01H\x01\x00\x01D\x03\x03\x96\xD0uu\x19\x14\xFC-                                                                               1
A\x00\

In [232]:
logs_df['Status Code'].value_counts()

301    1948
400    1658
200    1605
404     486
302      12
405      11
206      10
204       4
304       1
500       1
Name: Status Code, dtype: int64

In [233]:
a=logs_df.Endpoint.groupby(logs_df.Host).count().sort_values(ascending=False)

In [234]:
a

Host
1597610172    549
2050150420    282
3079201056    198
3240763672    197
3260520401    196
             ... 
1806363924      1
1806363153      1
1806363146      1
1806362897      1
2059537226      1
Name: Endpoint, Length: 765, dtype: int64

In [235]:
logs_df['Status Code'].value_counts()

301    1948
400    1658
200    1605
404     486
302      12
405      11
206      10
204       4
304       1
500       1
Name: Status Code, dtype: int64

In [236]:
logs_df['No of Requests']=logs_df.groupby(["Host"])['Endpoint'].transform('count')

/tmp/ipykernel_185/1435016481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_df['No of Requests']=logs_df.groupby(["Host"])['Endpoint'].transform('count')


In [237]:
logs_df.head(10)

,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size,No of Requests
0,2415145644,16/Feb/2023,03:28:45,GET,/config/getuser?index=0,HTTP/1.1,400,248.0,62
1,2757421940,16/Feb/2023,04:11:34,GET,/,HTTP/1.1,200,5952.0,5
2,1123632510,16/Feb/2023,04:39:52,GET,/robots.txt,HTTP/1.1,404,146.0,20
3,1123632510,16/Feb/2023,04:39:52,GET,/assets/img/favicon.png,HTTP/1.1,200,491.0,20
4,3118324652,16/Feb/2023,05:06:47,GET,/.git/config,HTTP/1.1,404,548.0,1
5,1123632485,16/Feb/2023,05:24:52,GET,/apim/devportal/site/public/images/logo.svg,HTTP/1.1,304,0.0,3
6,883396698,16/Feb/2023,06:26:16,GET,/,HTTP/1.1,200,5952.0,19
7,2556019923,16/Feb/2023,07:00:15,GET,/vendor/phpunit/phpunit/src/Util/PHP/eval-stdi...,HTTP/1.1,404,548.0,84
8,1920435509,16/Feb/2023,07:01:52,GET,/robots.txt,HTTP/1.1,404,146.0,1
9,3323217250,16/Feb/2023,07:26:00,GET,/,HTTP/1.1,200,5952.0,10


In [238]:
from sklearn.preprocessing import LabelEncoder

In [239]:
pd.DataFrame.to_csv(logs_df,'logs_df.csv',index=0)

# Label Encoding the features

In [240]:
le=LabelEncoder()

In [241]:
cols='Method','Status Code','Protocol','Endpoint'

In [242]:
for i in cols:
    logs_df[i]=le.fit_transform(logs_df[i])

/tmp/ipykernel_185/1174627788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_df[i]=le.fit_transform(logs_df[i])
/tmp/ipykernel_185/1174627788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_df[i]=le.fit_transform(logs_df[i])
/tmp/ipykernel_185/1174627788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [243]:
logs_df

,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size,No of Requests
0,2415145644,16/Feb/2023,03:28:45,4,236,6,6,248.0,62
1,2757421940,16/Feb/2023,04:11:34,4,10,6,0,5952.0,5
2,1123632510,16/Feb/2023,04:39:52,4,400,6,7,146.0,20
3,1123632510,16/Feb/2023,04:39:52,4,177,6,0,491.0,20
4,3118324652,16/Feb/2023,05:06:47,4,40,6,7,548.0,1
...,...,...,...,...,...,...,...,...,...
5731,2584535982,19/Feb/2023,02:48:08,7,240,6,7,146.0,6
5732,2584535982,19/Feb/2023,02:48:10,4,240,6,7,146.0,6
5733,2584535982,19/Feb/2023,02:48:12,7,10,6,8,150.0,6
5734,2584535982,19/Feb/2023,02:48:13,7,240,6,7,146.0,6


In [244]:
logs_df['Status Code'].value_counts()

3    1948
6    1658
0    1605
7     486
4      12
8      11
2      10
1       4
5       1
9       1
Name: Status Code, dtype: int64

In [245]:
import datetime

In [246]:
logs_df


,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size,No of Requests
0,2415145644,16/Feb/2023,03:28:45,4,236,6,6,248.0,62
1,2757421940,16/Feb/2023,04:11:34,4,10,6,0,5952.0,5
2,1123632510,16/Feb/2023,04:39:52,4,400,6,7,146.0,20
3,1123632510,16/Feb/2023,04:39:52,4,177,6,0,491.0,20
4,3118324652,16/Feb/2023,05:06:47,4,40,6,7,548.0,1
...,...,...,...,...,...,...,...,...,...
5731,2584535982,19/Feb/2023,02:48:08,7,240,6,7,146.0,6
5732,2584535982,19/Feb/2023,02:48:10,4,240,6,7,146.0,6
5733,2584535982,19/Feb/2023,02:48:12,7,10,6,8,150.0,6
5734,2584535982,19/Feb/2023,02:48:13,7,240,6,7,146.0,6


In [247]:
date=pd.to_datetime(logs_df.Date)

In [248]:
date

0      2023-02-16
1      2023-02-16
2      2023-02-16
3      2023-02-16
4      2023-02-16
          ...    
5731   2023-02-19
5732   2023-02-19
5733   2023-02-19
5734   2023-02-19
5735   2023-02-19
Name: Date, Length: 5736, dtype: datetime64[ns]

In [249]:
logs_df.Date=date

/tmp/ipykernel_185/3120451882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_df.Date=date


In [250]:
logs_df

,Host,Date,Time,Method,Endpoint,Protocol,Status Code,Content Size,No of Requests
0,2415145644,2023-02-16,03:28:45,4,236,6,6,248.0,62
1,2757421940,2023-02-16,04:11:34,4,10,6,0,5952.0,5
2,1123632510,2023-02-16,04:39:52,4,400,6,7,146.0,20
3,1123632510,2023-02-16,04:39:52,4,177,6,0,491.0,20
4,3118324652,2023-02-16,05:06:47,4,40,6,7,548.0,1
...,...,...,...,...,...,...,...,...,...
5731,2584535982,2023-02-19,02:48:08,7,240,6,7,146.0,6
5732,2584535982,2023-02-19,02:48:10,4,240,6,7,146.0,6
5733,2584535982,2023-02-19,02:48:12,7,10,6,8,150.0,6
5734,2584535982,2023-02-19,02:48:13,7,240,6,7,146.0,6
